<a href="https://colab.research.google.com/github/ErickMartinezz/AprendizajeAutomatico2/blob/main/ErickMartinez_EleccionDeAutomoviles_PP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 30px;
    background: rgba(0, 0, 30, 0.8);
    border-radius: 20px;
    box-shadow: 0 0 40px #ff0000;
    width: 90%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h1 style="
        text-align: center;
        font-size: 32px;
        color: #ff0000;
        text-shadow: 0 0 15px #ff0000;
    ">🚗 Aplicación Avanzada: Car Evaluation</h1>

    <p style="font-size:18px; line-height:1.6;">
        En este proyecto analizaremos el dataset <b>Car Evaluation</b> utilizando técnicas de clustering y clasificación:
        <b>KNN</b>, <b>RandomForest</b> y <b>Stacking Classifier</b>.
    </p>

    <p style="font-size:18px; line-height:1.6;">
        <b>Dataset elegido:</b>
        <a href="http://archive.ics.uci.edu/dataset/19/car+evaluation" style="color:#00ffff;">Car Evaluation (UCI)</a>
    </p>

    <h2 style="color:#ff0000; text-shadow: 0 0 10px #ff0000;">🎯 Objetivos</h2>
    <ul style="font-size:18px; line-height:1.6;">
        <li><b>Empresa:</b> Desarrollar un modelo que permita clasificar y segmentar vehículos según su aceptabilidad,
            con el fin de optimizar la gestión de stock y la estrategia de ventas.</li>
        <li><b>Cliente:</b> Ofrecer una herramienta de recomendación personalizada de acuerdo a lo que el cliente busca,
            mejorando la experiencia de compra y reduciendo la incertidumbre en la elección del vehículo.</li>
    </ul>
</div>
""")



In [2]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 25px;
    background: rgba(0, 0, 30, 0.8);
    border-radius: 20px;
    box-shadow: 0 0 30px #00ffff;
    width: 80%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h2 style="
        text-align: center;
        font-size: 28px;
        color: #00ffff;
        text-shadow: 0 0 10px #00ffff;
    ">📋 Información Académica</h2>
    <p style="font-size:18px; line-height:1.6;">
        <b>Nombre Alumno:</b> Erick Martinez
    </p>
    <p style="font-size:18px; line-height:1.6;">
        <b>Docente:</b> Julio Paredes
    </p>
</div>
""")


In [3]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 30px;
    background: rgba(0, 0, 30, 0.85);
    border-radius: 20px;
    box-shadow: 0 0 35px #ff6600;
    width: 85%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h2 style="
        text-align: center;
        font-size: 28px;
        color: #ff6600;
        text-shadow: 0 0 12px #ff6600;
    ">🎯 Objetivo del Proyecto</h2>
    <p style="font-size:18px; line-height:1.6;">
        Clasificar autos según la variable <b>clase</b> (inaceptable, aceptable, bueno, muy bueno) usando:
    </p>
    <ul style="font-size:18px; line-height:1.6;">
        <li><b>KNN</b> (clasificador supervisado)</li>
        <li><b>Random Forest</b> (clasificador supervisado)</li>
        <li><b>StackingClassifier</b> (apilado de modelos)</li>
    </ul>
    <p style="font-size:18px; line-height:1.6;">
        Además, se calculará una valorización económica de las predicciones (<b>payoff matrix</b>) para comparar modelos según valor, no solo métricas.
    </p>
</div>
""")


#Librerias, impotacion de dataset


In [4]:
# @title
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')


np.random.seed(42)

In [5]:
# @title
!pip install ucimlrepo

In [6]:
# @title
from ucimlrepo import fetch_ucirepo

# fetch dataset
car_evaluation = fetch_ucirepo(id=19)

# data (as pandas dataframes)
X = car_evaluation.data.features
y = car_evaluation.data.targets

# metadata
print(car_evaluation.metadata)

# variable information
print(car_evaluation.variables)


{'uci_id': 19, 'name': 'Car Evaluation', 'repository_url': 'https://archive.ics.uci.edu/dataset/19/car+evaluation', 'data_url': 'https://archive.ics.uci.edu/static/public/19/data.csv', 'abstract': 'Derived from simple hierarchical decision model, this database may be useful for testing constructive induction and structure discovery methods.', 'area': 'Other', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1728, 'num_features': 6, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1988, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5JP48', 'creators': ['Marko Bohanec'], 'intro_paper': {'ID': 249, 'type': 'NATIVE', 'title': 'Knowledge acquisition and explanation for multi-attribute decision making', 'authors': 'M. Bohanec, V. Rajkovič', 'venue': '8th Intl Workshop on Expert Systems and their Applications, 

In [7]:
df = pd.concat([X, y], axis=1)


#EDA

In [8]:
print('\n--- Descripción rápida ---')
display(df.describe(include='all').T)




--- Descripción rápida ---


,count,unique,top,freq
buying,1728,4,vhigh,432
maint,1728,4,vhigh,432
doors,1728,4,2,432
persons,1728,3,2,576
lug_boot,1728,3,small,576
safety,1728,3,low,576
class,1728,4,unacc,1210


In [9]:
# @title
# Renombrar las columnas a español para mayor claridad
df.rename(columns={
    'buying': 'precio_compra',
    'maint': 'precio_mantenimiento',
    'doors': 'puertas',
    'persons': 'personas',
    'lug_boot': 'maletero',
    'safety': 'seguridad',
    'class': 'clase'
}, inplace=True)

# Mostrar las primeras filas del DataFrame con los nombres de columnas actualizados
display(df.head())

,precio_compra,precio_mantenimiento,puertas,personas,maletero,seguridad,clase
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [10]:
# @title
# Renombrar las clases
df['clase'] = df['clase'].replace({
    'unacc': 'inaceptable',
    'acc': 'aceptable',
    'good': 'bueno',
    'vgood': 'muy bueno'
})

# Mostrar la distribución de las clases renombradas
display(df['clase'].value_counts())

,count
clase,
inaceptable,1210
aceptable,384
bueno,69
muy bueno,65


In [11]:
# @title
import plotly.express as px

# Gráfico de distribución por clase
fig = px.histogram(df, x='clase',
                   color='clase',
                   title='🚗 Distribución de Clases de Autos',
                   color_discrete_sequence=px.colors.sequential.Tealgrn,
                   labels={'clase':'Clase de Auto'},
                   text_auto=True)  # muestra los conteos sobre las barras

# Estilo automotriz
fig.update_layout(
    title_font_size=24,
    title_font_color='darkred',
    xaxis_title='Clase',
    yaxis_title='Cantidad de Autos',
    plot_bgcolor='lightgray',  # fondo gris tipo pista
    paper_bgcolor='whitesmoke',
    font=dict(family='Arial', size=14, color='black'),
    showlegend=False
)

fig.show()


In [12]:
# @title
import plotly.express as px

# Columnas categóricas a graficar
categorical_cols = ['precio_compra', 'precio_mantenimiento', 'seguridad']

for c in categorical_cols:
    fig = px.histogram(df, x=c,
                       color=c,
                       title=f'🚗 Distribución: {c}',
                       color_discrete_sequence=px.colors.sequential.Tealgrn,
                       text_auto=True,
                       category_orders={c: sorted(df[c].unique())})

    fig.update_layout(
        title_font_size=20,
        title_font_color='darkred',
        xaxis_title=c.replace('_', ' ').capitalize(),
        yaxis_title='Cantidad de Autos',
        plot_bgcolor='lightgray',
        paper_bgcolor='whitesmoke',
        font=dict(family='Arial', size=14, color='black'),
        showlegend=False
    )

    fig.show()


In [13]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 25px;
    background: rgba(0, 0, 30, 0.85);
    border-radius: 20px;
    box-shadow: 0 0 35px #ffcc00;
    width: 85%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h2 style="
        text-align: center;
        font-size: 28px;
        color: #ffcc00;
        text-shadow: 0 0 12px #ffcc00;
    ">📊 Visualización y Análisis Exploratorio</h2>
    <p style="font-size:18px; line-height:1.6;">
        Se transformaron todas las variables al español y se generaron gráficos interactivos.
        En primera instancia, se observa la distribución de los autos según las clases,
        y posteriormente se comparan algunas variables categóricas relevantes.
    </p>
</div>
""")


# Armado de modelo

##Preparar datos para clasificación

In [14]:
# @title
# X = features, y = clase
X = pd.get_dummies(df.drop(columns=['clase']))  # codificación rápida
y = df['clase']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)


#Modelo 1: Random KNN


In [15]:
# @title
# Escalar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

y_pred_knn = knn.predict(X_test_scaled)
print('KNN Accuracy:', accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))


KNN Accuracy: 0.8634259259259259
              precision    recall  f1-score   support

   aceptable       0.79      0.71      0.75        96
       bueno       0.56      0.53      0.55        17
 inaceptable       0.90      0.97      0.94       303
   muy bueno       0.50      0.06      0.11        16

    accuracy                           0.86       432
   macro avg       0.69      0.57      0.58       432
weighted avg       0.85      0.86      0.85       432



#Modelo 2: Random Forest

In [16]:
# @title
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
print('Random Forest Accuracy:', accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.9699074074074074
              precision    recall  f1-score   support

   aceptable       0.91      0.97      0.94        96
       bueno       1.00      0.88      0.94        17
 inaceptable       0.99      0.98      0.99       303
   muy bueno       0.93      0.88      0.90        16

    accuracy                           0.97       432
   macro avg       0.96      0.93      0.94       432
weighted avg       0.97      0.97      0.97       432



#Modelo 3: Stacking Classifier

In [17]:
# @title
stack = StackingClassifier(
    estimators=[
        ('knn', KNeighborsClassifier(n_neighbors=5)),
        ('rf', RandomForestClassifier(n_estimators=150, random_state=42)),
        ('lr', LogisticRegression(max_iter=1000))
    ],
    final_estimator=LogisticRegression(max_iter=1000),
    cv=5,
    n_jobs=-1
)

stack.fit(X_train_scaled, y_train)
y_pred_stack = stack.predict(X_test_scaled)

print('Stacking Accuracy:', accuracy_score(y_test, y_pred_stack))
print(classification_report(y_test, y_pred_stack))


Stacking Accuracy: 0.9675925925925926
              precision    recall  f1-score   support

   aceptable       0.93      0.94      0.93        96
       bueno       1.00      0.94      0.97        17
 inaceptable       0.98      0.98      0.98       303
   muy bueno       0.89      1.00      0.94        16

    accuracy                           0.97       432
   macro avg       0.95      0.96      0.96       432
weighted avg       0.97      0.97      0.97       432



#Comparación de modelos

In [18]:
# @title
import plotly.express as px
import pandas as pd

# Resultados de los modelos
results = {
    'Modelo': ['KNN', 'RandomForest', 'Stacking'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_knn),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_stack)
    ],
    'F1_macro': [
        f1_score(y_test, y_pred_knn, average='macro'),
        f1_score(y_test, y_pred_rf, average='macro'),
        f1_score(y_test, y_pred_stack, average='macro')
    ]
}

res_df = pd.DataFrame(results)
res_df


,Modelo,Accuracy,F1_macro
0,KNN,0.863426,0.584711
1,RandomForest,0.969907,0.941299
2,Stacking,0.967593,0.955912


In [19]:
# @title
fig = px.bar(res_df.melt(id_vars='Modelo', value_vars=['Accuracy','F1_macro']),
             x='Modelo', y='value', color='variable',
             barmode='group',
             text='value',
             labels={'value':'Valor', 'variable':'Métrica'},
             title='🚗 Comparación interactiva de modelos')

fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.update_layout(template='plotly_dark', yaxis=dict(range=[0,1]))
fig.show()


In [20]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 25px;
    background: rgba(0, 0, 30, 0.85);
    border-radius: 20px;
    box-shadow: 0 0 40px #ff9900;
    width: 85%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h2 style="
        text-align: center;
        font-size: 28px;
        color: #ff9900;
        text-shadow: 0 0 12px #ff9900;
    ">📊 Resumen de Resultados de Modelos</h2>
    <p style="font-size:18px; line-height:1.6;">
        Análisis breve de los resultados obtenidos para los tres modelos de clasificación:
    </p>
    <ul style="font-size:17px; line-height:1.6;">
        <li><b>KNN:</b> Accuracy ≈ 0.863, F1_macro ≈ 0.585. Desempeño correcto, pero con dificultades para balancear clases minoritarias.</li>
        <li><b>Random Forest:</b> Accuracy ≈ 0.970, F1_macro ≈ 0.941. Excelente desempeño general y alta precisión para la mayoría de clases.</li>
        <li><b>Stacking Classifier:</b> Accuracy ≈ 0.968, F1_macro ≈ 0.956. Combina varios modelos; mejora la medida balanceada (F1_macro), aunque la Accuracy es ligeramente menor que Random Forest.</li>
    </ul>
    <p style="font-size:18px; line-height:1.6;">
        <b>Conclusión:</b> Random Forest tiene la mayor precisión global, mientras que Stacking Classifier ofrece mejor balance entre clases. KNN sirve como referencia simple.
    </p>
</div>
""")


# **Conclusion**

In [21]:
# @title
from IPython.display import HTML

HTML("""
<div style="
    margin: 20px auto;
    padding: 30px;
    background: rgba(0, 0, 30, 0.9);
    border-radius: 20px;
    box-shadow: 0 0 45px #ff4444;
    width: 85%;
    font-family: 'Trebuchet MS', sans-serif;
    color: white;
">
    <h2 style="
        text-align: center;
        font-size: 28px;
        color: #ff4444;
        text-shadow: 0 0 12px #ff4444;
    ">🏁 Conclusión Final del TP1 - PP2</h2>
    <p style="font-size:18px; line-height:1.6;">
        Realicé un trabajo explorando datos de automóviles, intentando predecir y diferenciar en 4 clases,
        entrenando distintos modelos de clasificación.
    </p>
    <p style="font-size:18px; line-height:1.6;">
        Como resultado final, se concluyó que el modelo que mejor se adaptó fue <b>Random Forest</b>.
    </p>
    <p style="font-size:18px; line-height:1.6;">
        Una vez creado todo el código y los modelos, se ajustaron los marcadores y estilos para presentar
        un trabajo visualmente más atractivo, utilizando la ayuda de ChatGPT para generar HTML y displays interactivos.
    </p>
    <p style="font-size:18px; line-height:1.6;">
        Para próximas entregas se planea realizar una valorización del modelo, intentando predecir cuál es la mejor opción
        en términos prácticos y económicos.
    </p>
</div>
""")



#DESDE ACA DEBE TERMINAR TP2_ CON LA PARTE ECNÓMICA + conclusiones interactivas !!!

In [22]:
# @title
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets

# --- 🎨 ENCABEZADO HTML ---
display(HTML("""
<div style="
    background: linear-gradient(135deg, #0a0a0a, #1b1b1b);
    border: 2px solid #ff0000;
    border-radius: 20px;
    padding: 25px;
    color: white;
    box-shadow: 0 0 25px #ff0000;
    font-family: 'Trebuchet MS', sans-serif;
    text-align: center;
">
  <h1 style="color:#ff3333; text-shadow:0 0 15px #ff0000;">🚗 Evaluador Inteligente de Automóviles</h1>
  <p style="font-size:17px; line-height:1.5;">
     Introduce las características del vehículo para conocer su <b>nivel de aceptabilidad</b> según el modelo <b>Random Forest</b>.
  </p>
  <hr style="border:1px solid #ff0000; width:80%;">
</div>
"""))

# --- 🧠 FUNCIÓN DE PREDICCIÓN ---
columnas_modelo = X.columns

def predecir_auto(precio_compra, precio_mantenimiento, puertas, personas, maletero, seguridad):
    datos_nuevos = pd.DataFrame({
        'precio_compra': [precio_compra],
        'precio_mantenimiento': [precio_mantenimiento],
        'puertas': [puertas],
        'personas': [personas],
        'maletero': [maletero],
        'seguridad': [seguridad]
    })
    datos_nuevos_encoded = pd.get_dummies(datos_nuevos)
    datos_nuevos_encoded = datos_nuevos_encoded.reindex(columns=columnas_modelo, fill_value=0)
    pred = rf.predict(datos_nuevos_encoded)
    return pred[0]

# --- 💬 INTERPRETACIÓN ---
def interpretar_prediccion(prediccion):
    if prediccion == 'inaceptable':
        color = "#ff3333"
        emoji = "❌"
        texto = "Auto no recomendado. Poca seguridad o mala relación costo-beneficio."
    elif prediccion == 'aceptable':
        color = "#ffaa00"
        emoji = "⚠️"
        texto = "Vehículo aceptable, pero se recomienda comparar otras opciones."
    elif prediccion == 'bueno':
        color = "#33ff77"
        emoji = "✅"
        texto = "Buena opción de compra con equilibrio entre costo y rendimiento."
    elif prediccion == 'muy bueno':
        color = "#00ffff"
        emoji = "🌟"
        texto = "Excelente elección: alta seguridad y gran relación calidad/precio."
    else:
        color = "white"
        emoji = "❓"
        texto = "Resultado no reconocido."
    return emoji, color, texto


# --- 🎛️ WIDGETS ---
precio_compra = widgets.Dropdown(options=['vhigh', 'high', 'med', 'low'], description='💰 Compra:')
precio_mantenimiento = widgets.Dropdown(options=['vhigh', 'high', 'med', 'low'], description='🛠️ Mantenimiento:')
puertas = widgets.Dropdown(options=['2', '3', '4', '5more'], description='🚪 Puertas:')
personas = widgets.Dropdown(options=['2', '4', 'more'], description='👥 Personas:')
maletero = widgets.Dropdown(options=['small', 'med', 'big'], description='🧳 Maletero:')
seguridad = widgets.Dropdown(options=['low', 'med', 'high'], description='🛡️ Seguridad:')

boton_predecir = widgets.Button(
    description='🔍 Evaluar Auto',
    button_style='danger',
    layout=widgets.Layout(width='50%', height='40px')
)

salida = widgets.Output()

# --- ⚙️ FUNCIÓN DEL BOTÓN ---
def al_hacer_click(b):
    with salida:
        clear_output()
        pred = predecir_auto(precio_compra.value, precio_mantenimiento.value, puertas.value, personas.value, maletero.value, seguridad.value)
        emoji, color, texto = interpretar_prediccion(pred)

        display(HTML(f"""
        <div style="
            background: rgba(20,20,20,0.9);
            border: 2px solid {color};
            border-radius: 15px;
            box-shadow: 0 0 25px {color};
            margin-top: 20px;
            padding: 20px;
            color: white;
            font-family: 'Trebuchet MS';
        ">
            <h2 style="color:{color};">{emoji} Resultado: {pred.upper()}</h2>
            <p style="font-size:17px;">{texto}</p>
        </div>
        """))

boton_predecir.on_click(al_hacer_click)

# --- 📊 MOSTRAR TODO ---
display(
    widgets.VBox([
        widgets.HBox([precio_compra, precio_mantenimiento]),
        widgets.HBox([puertas, personas]),
        widgets.HBox([maletero, seguridad]),
        boton_predecir,
        salida
    ])
)


In [23]:
# @title 🏢 Valor para la Empresa (Concesionaria)
from IPython.display import HTML, display

display(HTML("""
<div style="
    background: linear-gradient(135deg, #0a0a0a, #1c1c1c);
    border: 2px solid #ff0000;
    border-radius: 20px;
    box-shadow: 0 0 25px #ff0000;
    padding: 25px;
    color: white;
    font-family: 'Trebuchet MS', sans-serif;
">
    <h2 style="color:#ff3333; text-shadow:0 0 15px #ff0000;">🏢 Impacto Empresarial: Optimización de Stock y Estrategia de Ventas</h2>
    <p style="font-size:17px; line-height:1.6;">
        El modelo de clasificación permite a la concesionaria <b>evaluar objetivamente cada vehículo</b> según su nivel de aceptabilidad,
        basándose en características técnicas y de seguridad.
    </p>
    <p style="font-size:17px; line-height:1.6;">
        Con estos resultados, la empresa puede:
        <ul style="font-size:17px; line-height:1.6;">
            <li>🔸 Priorizar la <b>adquisición o exhibición</b> de modelos con alta aceptación (bueno o muy bueno).</li>
            <li>🔸 <b>Segmentar el stock</b> para mejorar la gestión de ventas y rotación.</li>
            <li>🔸 Diseñar <b>estrategias publicitarias basadas en datos</b>, focalizando en los segmentos con mejor retorno.</li>
        </ul>
    </p>
    <p style="font-size:17px; line-height:1.6;">
        En definitiva, el sistema transforma la información del dataset en una <b>herramienta de apoyo a la decisión comercial</b>,
        permitiendo tomar decisiones más rápidas y rentables.
    </p>
</div>
"""))


In [24]:
# @title
from sklearn.metrics import confusion_matrix

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_rf, labels=rf.classes_)
total_aciertos = np.trace(cm)
total_errores = cm.sum() - total_aciertos

# Supuestos de impacto económico
valor_acierto = 400    # beneficio promedio por predicción correcta
costo_error = -200     # pérdida o costo por error

impacto_total = (total_aciertos * valor_acierto) + (total_errores * costo_error)
precision_modelo = accuracy_score(y_test, y_pred_rf)

display(HTML(f"""
<div style="
    background: linear-gradient(135deg, #111, #222);
    border-radius: 20px;
    padding: 25px;
    color: white;
    box-shadow: 0 0 30px #ff0000;
    font-family: 'Trebuchet MS';
    margin-top: 15px;
">
  <h2 style="color:#ff3333;">📈 Valor Comercial del Modelo (Concesionaria)</h2>
  <p style="font-size:17px; line-height:1.6;">
    <b>Precisión del modelo:</b> {precision_modelo:.2%}<br>
    <b>Predicciones correctas:</b> {total_aciertos}<br>
    <b>Errores:</b> {total_errores}<br>
  </p>
  <p style="font-size:18px; color:#00ff99;">
    💰 <b>Impacto económico estimado:</b> USD {impacto_total:,.0f}
  </p>
  <p style="font-size:15px; color:#ccc;">
    *(Se estima una ganancia de USD {valor_acierto} por recomendación acertada y una pérdida de USD {abs(costo_error)} por error.)*
  </p>
</div>
"""))


In [25]:
# @title 👥 Valor para el Cliente (Usuario Final)
from IPython.display import HTML, display

display(HTML("""
<div style="
    background: linear-gradient(135deg, #0a0a0a, #1c1c1c);
    border: 2px solid #00ffff;
    border-radius: 20px;
    box-shadow: 0 0 25px #00ffff;
    padding: 25px;
    color: white;
    font-family: 'Trebuchet MS', sans-serif;
">
    <h2 style="color:#00ffff; text-shadow:0 0 15px #00ffff;">👥 Impacto en el Cliente: Recomendación Personalizada</h2>
    <p style="font-size:17px; line-height:1.6;">
        Desde la perspectiva del comprador, el modelo se convierte en un <b>asistente inteligente</b> capaz de recomendar vehículos según las preferencias del usuario.
    </p>
    <p style="font-size:17px; line-height:1.6;">
        Por ejemplo, un cliente que busca un auto <b>económico, seguro y de bajo mantenimiento</b> puede recibir sugerencias en tiempo real,
        basadas en datos objetivos del modelo.
    </p>
    <ul style="font-size:17px; line-height:1.6;">
        <li>🔹 Aumenta la <b>transparencia</b> en la elección del vehículo.</li>
        <li>🔹 Reduce la posibilidad de una compra insatisfactoria.</li>
        <li>🔹 Promueve la <b>confianza del cliente</b> y su fidelización.</li>
    </ul>
    <p style="font-size:17px;">
        De esta forma, el sistema no solo clasifica autos, sino que <b>mejora la experiencia de compra</b> a través de la personalización y la tecnología.
    </p>
</div>
"""))


In [26]:
# @title
# Supuestos de valor percibido por el usuario
ahorro_promedio = 250       # ahorro por elegir un vehículo adecuado
costo_error_cliente = -100  # costo de una mala elección

impacto_cliente = (total_aciertos * ahorro_promedio) + (total_errores * costo_error_cliente)

display(HTML(f"""
<div style="
    background: linear-gradient(135deg, #111, #222);
    border-radius: 20px;
    padding: 25px;
    color: white;
    box-shadow: 0 0 30px #00ffff;
    font-family: 'Trebuchet MS';
    margin-top: 15px;
">
  <h2 style="color:#00ffff;">💡 Valor Percibido por el Cliente</h2>
  <p style="font-size:17px; line-height:1.6;">
    <b>Aciertos del modelo:</b> {total_aciertos}<br>
    <b>Errores:</b> {total_errores}<br>
    <b>Beneficio medio por acierto:</b> USD {ahorro_promedio}<br>
    <b>Pérdida estimada por error:</b> USD {abs(costo_error_cliente)}<br>
  </p>
  <p style="font-size:18px; color:#00ff99;">
    🌟 <b>Valor total estimado para los clientes:</b> USD {impacto_cliente:,.0f}
  </p>
  <p style="font-size:15px; color:#ccc;">
    *(El modelo ayuda al usuario a ahorrar al elegir autos más seguros, eficientes y convenientes.)*
  </p>
</div>
"""))


In [27]:
# @title 🏁 Conclusión General del Modelo - Car Evaluation

from IPython.display import HTML, display

display(HTML("""
<div style="
    background: linear-gradient(135deg, #000000, #1a1a1a);
    border-radius: 20px;
    padding: 35px;
    color: white;
    box-shadow: 0 0 40px #ff0000;
    font-family: 'Trebuchet MS', sans-serif;
    margin-top: 25px;
">
  <h2 style="color:#ff0000; text-align:center;">🏁 Conclusión General del Modelo</h2>
  <p style="font-size:17px; line-height:1.7;">
    El modelo desarrollado a partir del dataset <b>Car Evaluation</b> permitió construir un sistema de
    <b>clasificación inteligente</b> capaz de evaluar la aceptabilidad de un vehículo con alta precisión.
    Mediante técnicas como <b>Random Forest</b>, se logró interpretar variables clave como el
    <b>precio de compra, mantenimiento, seguridad, capacidad y confort</b>,
    generando una predicción confiable del nivel de calidad del automóvil.
  </p>

  <p style="font-size:17px; line-height:1.7;">
    Desde el punto de vista empresarial, este modelo representa una herramienta estratégica que
    contribuye a <b>optimizar la gestión del stock</b>, planificar mejor la oferta y
    <b>reducir pérdidas por decisiones de compra ineficientes</b>.
    A su vez, para el cliente, ofrece una <b>recomendación personalizada</b> que
    mejora la experiencia de compra, reduce riesgos y promueve una decisión más informada.
  </p>

  <p style="font-size:17px; line-height:1.7;">
    En conjunto, el sistema se consolida como un ejemplo práctico de cómo la <b>Inteligencia Artificial</b>
    puede agregar valor en contextos comerciales reales,
    transformando datos históricos en <b>conocimiento útil para la toma de decisiones</b>.
  </p>

  <p style="text-align:center; font-size:18px; margin-top:20px; color:#00ffcc;">
    🚗 Innovación, datos y aprendizaje automático al servicio del futuro automotriz.
  </p>
</div>
"""))


In [28]:
# @title 🚀 Posibles Mejoras y Escalabilidad Futura

from IPython.display import HTML, display

display(HTML("""
<div style="
    background: linear-gradient(135deg, #0a0a0a, #1a1a1a);
    border-radius: 20px;
    padding: 35px;
    color: white;
    box-shadow: 0 0 40px #00ffff;
    font-family: 'Trebuchet MS', sans-serif;
    margin-top: 25px;
">
  <h2 style="color:#00ffff; text-align:center;">🚀 Posibles Mejoras y Escalabilidad Futura</h2>

  <p style="font-size:17px; line-height:1.7;">
    Este proyecto puede evolucionar hacia un sistema más robusto y con mayor alcance práctico.
    A continuación, se detallan algunas posibles líneas de mejora y expansión tecnológica:
  </p>

  <ul style="font-size:16px; line-height:1.8; margin-left:25px;">
    <li>📊 <b>Integración de más variables</b>: incorporar datos reales como consumo, emisiones,
    marca o año del vehículo para enriquecer las predicciones.</li>

    <li>🌐 <b>Implementación web</b>: desplegar el modelo en una aplicación interactiva
    utilizando <b>Streamlit</b> o <b>Flask</b> para ofrecer recomendaciones online a usuarios y concesionarias.</li>

    <li>🤖 <b>Optimización del modelo</b>: explorar algoritmos adicionales como <b>XGBoost</b> o
    <b>SVM</b>, y realizar una búsqueda de hiperparámetros más exhaustiva para mejorar la precisión.</li>

    <li>💡 <b>Sistema de recomendación inteligente</b>: permitir que el usuario ingrese sus
    preferencias (precio, tamaño, mantenimiento, etc.) y obtener sugerencias personalizadas de vehículos.</li>

    <li>📱 <b>Escalabilidad y conexión con bases externas</b>: enlazar el modelo con catálogos
    o APIs de precios automotrices para mantener la información actualizada en tiempo real.</li>
  </ul>

  <p style="font-size:17px; line-height:1.7; margin-top:20px;">
    Estas mejoras permitirían transformar el prototipo actual en una <b>herramienta predictiva integral</b>,
    útil tanto para la industria automotriz como para los consumidores finales,
    consolidando el valor del análisis de datos aplicado al sector.
  </p>

  <p style="text-align:center; font-size:18px; margin-top:25px; color:#00ffcc;">
    ⚙️ Datos, aprendizaje automático y visión de futuro para un mercado automotriz más inteligente.
  </p>
</div>
"""))
